<h1>Exploração dos dados na camada silver</h1>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName('twitter_silver')\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/01/08 20:20:01 WARN Utils: Your hostname, PcArthur resolves to a loopback address: 127.0.1.1; using 172.23.223.44 instead (on interface eth0)
25/01/08 20:20:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/08 20:21:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_tweet = spark.read.json('/home/arthur/pipeline_tweets/datalake/silver/twitter_datascience/tweet')

In [3]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [4]:
from pyspark.sql import functions as f 

In [5]:
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        ).withColumn("weekday", f.date_format("created_date", "E"))

In [6]:
tweet_conversas.show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2025-01-05|       9|   518|    591|    508|      472|    Sun|
|  2025-01-07|      22|  1716|   1813|   1359|     1631|    Tue|
|  2025-01-06|      41|  2322|   2642|   2889|     2630|    Mon|
+------------+--------+------+-------+-------+---------+-------+

